In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# Get to the folder we are at
FOLDERNAME = 'Colab\ Notebooks/Kaggle'
%cd drive/MyDrive/$FOLDERNAME/

In [ ]:
import os
import shutil
import glob

def make_if_dont_exist(folder_path,overwrite=False):
  if os.path.exists(folder_path):
    if not overwrite:
        print(f"{folder_path} exists.")
    else:
        print(f"{folder_path} overwritten")
        shutil.rmtree(folder_path)
        os.makedirs(folder_path)

  else:
    os.makedirs(folder_path)
    print(f"{folder_path} created!")

In [ ]:
mount_dir = os.getcwd() + "/nnU-Net"
base_dir = os.getcwd() + "/nnU-Net"

# Configure environment variables
print("Current Working Directory {}".format(os.getcwd()))
path_dict = {
    "nnUNet_raw" : os.path.join(mount_dir, "nnUNet_raw"),
    "nnUNet_preprocessed" : os.path.join(mount_dir, "nnUNet_preprocessed"), # 1 experiment: 1 epoch took 112s
    "nnUNet_results" : os.path.join(mount_dir, "nnUNet_results")
}

# Write paths to environment variables
for env_var, path in path_dict.items():
  os.environ[env_var] = path

# Check whether all environment variables are set correct!
for env_var, path in path_dict.items():
  if os.getenv(env_var) != path:
    print("Error:")
    print("Environment Variable {} is not set correctly!".format(env_var))
    print("Should be {}".format(path))
    print("Variable is {}".format(os.getenv(env_var)))
  make_if_dont_exist(path, overwrite=False)

os.chdir(base_dir)

In [ ]:
!pip install -q nnunetv2
import nnunetv2

In [ ]:
# command for pre-process and plan for training
!nnUNetv2_plan_and_preprocess -d 780 -c 3d_fullres

In [ ]:
# command for training nnU-Net
!nnUNetv2_train --c --npz 780 3d_fullres 0

In [ ]:
# command to find best config after completing the training
# this commant will print the command for inference and post-process in the end
!nnUNetv2_find_best_configuration 780 -c 3d_fullres -f 0

In [ ]:
# inference command
!nnUNetv2_predict -d Dataset780_Cholecystitis -i "nnUNet_inference/Cholecystitis_val200_ts1_ts2" -o "nnUNet_inference/Cholecystitis_val200_ts1_ts2_seg_result" -f 0 -tr nnUNetTrainer -c 3d_fullres -p nnUNetPlans

In [ ]:
# post-process command
!nnUNetv2_apply_postprocessing -i "nnUNet_inference/Cholecystitis_val200_ts1_ts2_seg_result" -o "nnUNet_inference/Cholecystitis_val200_ts1_ts2_seg_result_pp" -pp_pkl_file "/content/drive/MyDrive/Colab Notebooks/Kaggle/nnU-Net/nnUNet_results/Dataset780_Cholecystitis/nnUNetTrainer__nnUNetPlans__3d_fullres/crossval_results_folds_0/postprocessing.pkl" -np 8 -plans_json "/content/drive/MyDrive/Colab Notebooks/Kaggle/nnU-Net/nnUNet_results/Dataset780_Cholecystitis/nnUNetTrainer__nnUNetPlans__3d_fullres/crossval_results_folds_0/plans.json"